In [6]:
import os
import re
import random
import numpy as np

from openai import OpenAI
from settings.constants import *
from utils.funcs import *

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE
)

In [3]:
def chat_completion_request_openai(prompt: str = ""):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        temperature=1.0,
        max_tokens=1500
    )
    if response.choices:
        return response.choices[0].message.content
    return None


def get_draft_answer_critique(question, draft_answer):
    prompt = (
        f"Question: {question}"
        f"Draft Answer: {draft_answer}"
        "Please critique the draft answer."
        "Carefully assess if the answer is correct of not and why."
        "Consider multiple ways of verifying if the answer is correct."
        "DO: Point out EVERY flaw and hold the draft answer to a very high standard."
        "DO: Provide specific recommendations to improve the answer."
        "DO: Think step by step."
        "DO NOT provide a revised answer"
    )
    return chat_completion_request_openai(prompt)


def get_improved_answer(question, draft_answer, critique):
    prompt = (
        f"Question: {question}"
        f"Draft Answer: {draft_answer}"
        f"Critique: {critique}"
        "Please improve the draft answer based on the critique following this format:"
        "Reasoning Process: <step-by-ste- reasoning process>\n"
        "Verification: <verification of the facts>\n"
        "Final Answer: <the improved and verified answer>\n"
    )
    return chat_completion_request_openai(prompt)

def rate_answer(question, answer):
    prompt = (
        f"Question: {question}"
        f"Answer: {answer}"
        "As an expert on this topic, please provide a detailed critique "
        "Provide ONLY a critique, not a suggested answer."
        "Then, rate the answer on a scale of 0 to 100."
        "The response should be in the following format:"
        "Critique: <detailed critique>\n"
        "Rating: <0 to 100 rating>\n"
    )
    response = chat_completion_request_openai(prompt)
    try: 
        match = re.search(r"Rating:\s+(\d+)", response)
        if match:
            rating = int(match.group(1))
            if rating > 95:
                rating = 95.0
            rating /= 100.0
        else:
            raise ValueError("Rating not found in response.")
    except Exception as e:
        print(f"Error extracting rating: {e}")
        print(f"Rating response was: {response}")
        rating = 0.0
    return rating


def get_answer_directly_from_llm(question):
    prompt = (
        f"Question: {question}\n"
        "Please provide the answer to this question with detailed reasoning. Follow this format:\n"
        "Reasoning process: <step-by-step reasoning process>\n"
        "Verification: <verification of facts>\n"
        "Final Answer: <the improved and verified answer>\n"
    )
    llm_response = chat_completion_request_openai(prompt)

    try:
        match = re.search(r"Final Answer:\s*(.*)", llm_response)
        final_answer = match.group(1).strip() if match else None
    except Exception as e:
        final_answer = None


In [5]:
prompt = "return an array of strings where each array is a season"
response = chat_completion_request_openai(prompt)
print(response)

Certainly! Below is an example of how you can represent an array of strings where each array corresponds to a season:

```json
{
  "seasons": {
    "spring": ["March", "April", "May"],
    "summer": ["June", "July", "August"],
    "fall": ["September", "October", "November"],
    "winter": ["December", "January", "February"]
  }
}
```

In this format, each key represents a season, and the value is an array of strings corresponding to the months typically associated with that season. If you are looking for something different, please let me know!


In [4]:
prompt = "What is the capital of France"
draft_answer = "The capital of France is Lyon"

critique = get_draft_answer_critique(prompt, draft_answer)
print(critique)

The draft answer states, "The capital of France is Lyon," which is incorrect. The accurate capital of France is Paris. Here are several points of critique regarding the draft answer:

1. **Factual Error**: The most glaring flaw in the draft is the assertion that Lyon is the capital. This is incorrect information that misleads the reader. 

2. **Verification Method**: To ensure accuracy, one could verify this information in various reliable sources including:
   - **Official Government Websites**: The French government’s website would list Paris as the capital.
   - **Encyclopedias**: Reputable encyclopedias, such as Britannica or similar platforms, would confirm that Paris is the capital.
   - **Geographical Texts**: Reference materials on geography or political science would also indicate Paris as the capital city.

3. **Clarity and Precision**: The answer lacks clarity in explaining why Lyon is mentioned in the context of France. Lyon is a significant city known for its historical an

In [5]:
prompt = "What is the capital of France"
draft_answer = "The capital of France is Lyon"
critique = "That is just wrong"

improved_answer = get_improved_answer(prompt, draft_answer, critique)
print(improved_answer)

Reasoning Process: The initial draft inaccurately identified Lyon as the capital of France. To correct this, I must first recall the correct capital city based on my knowledge of world geography and the status of cities in France. Paris is widely recognized as the capital and is the largest city in France, serving as the political, economic, and cultural center of the country.

Verification: I can verify that Paris has been the capital of France since 508 AD, and it is frequently referenced in geography textbooks, government documents, and even travel guides. Additionally, information from reputable sources like encyclopedias and official government websites confirms that Paris is the capital.

Final Answer: The capital of France is Paris.


In [6]:
question = "What is the capital of France"
answer = "The capital of France is Paris"
rating = rate_answer(question, answer)
print(f"Rating was: {rating}")

Rating was: 0.7


### Answer Improvement Rating Pipeline
For a given anser to a question:
1. Critique
2. Improve
3. Rate

In [10]:
seed_answers = [
    "I don't know the answer",
    "I'm not sure",
    "I can't say on that one"
]